#Lab.00 / IBM3202 - Installing Software on Google Colab for IBM3202 tutorials

While most software and applications required for the IBM3202 Molecular Modeling and Simulation course can be installed into Google Colab during the tutorials, some other software takes a significant amount of time between compilation and installation. To avoid consuming the limited time of the tutorials for these tasks, here we include the installation step for such programs.

The most common procedure comprises:
1. Compilation of the required software during a Google Colab session
2. Backing up the compiled software into your Google Drive
3. By the time the tutorials take place, these precompiled programs will be onto your Google Colab session

#**Before we start...**

##**1. Copy this notebook to your Drive, otherwise the installation will not work**

You can do this by clicking on the **Save a Copy to Drive** option on the **File** menu.

##**2. Watch out for idle timeouts on Google Colab**

To be able to offer free computational resources on the cloud, Colab has **idle timeouts** that disconnect the runtime session if left unattended for too long and **maximum lifetimes**. While the maximum lifetime on a standard Google Colab runtime can be as much as 12 hours, the **idle timeouts vary over time**. 

Thus, is is imperative that you keep an eye on your software installation every 10-15 min to avoid idle timeouts that would interrupt the whole process.

##**3. Where is the installed software located after backing up on Google Drive?**

All installed software will be located in a new **IBM3202** folder in your Google Drive account with the exception of **PyRosetta**, which will side in a **prefix** folder in your Google Drive account as per the [PyRosetta Google Drive Setup](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/01.01-PyRosetta-Google-Drive-Setup.ipynb) instructions from **PyRosetta Workshops**.

#Installation of PyRosetta

##Getting the PyRosetta License

First enter [the following link](https://els2.comotion.uw.edu/product/pyrosetta), fill the required information and press the buttons indicated by the black arrows to obtain a PyRosetta License

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_01.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_02.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_03.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_04.png'/>
</center>
</figure>

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_05.png'/>
</center>
</figure>

After this, you will receive an email containing a **username and password** required for obtaining PyRosetta.
<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pyrosetta_final.png' width=500/>
</center>
</figure>

##Setting up PyRosetta in Google Colab

To install PyRosetta, we will mount your Google Drive to be able to transfer files. You will be requested to authorize this Google Drive mounting by entering an authorization code into an input box. Copy the code, paste it into this box and then press 'Enter'.

In [ ]:
# Mounting Google Drive and add it to Python sys path  

google_drive_mount_point = '/content/google_drive'

import os, sys, time

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount(google_drive_mount_point)

google_drive = google_drive_mount_point + '/My Drive'
google_drive_prefix = google_drive + '/prefix'

if not os.path.isdir(google_drive_prefix):
    os.mkdir(google_drive_prefix)

pyrosetta_install_prefix_path = '/content/prefix'
if os.path.islink(pyrosetta_install_prefix_path):
    os.unlink(pyrosetta_install_prefix_path)
os.symlink(google_drive_prefix, pyrosetta_install_prefix_path)

sys.path.insert(0, pyrosetta_install_prefix_path)

In [ ]:
os.environ["PYTHONPATH"] = os.environ["PYTHONPATH"] + ':' + pyrosetta_install_prefix_path

In the following script, we will compile and install PyRosetta, and also back it up to Google Drive for future usage.

Please note that you have to **replace --user and --password** in this script with the one obtained through your PyRosetta license

⚠️**WARNING:** You might also need to change the release number of PyRosetta to the most up to date version, although we usually keep update this periodically.

In [ ]:
#sys.path.insert(0, google_drive_packages_path)

# installing PyRosetta
if sys.version_info.major != 3 or sys.version_info.minor != 6:
    print('Need Python-3.6 to run!')
    sys.exit(1)

import_start_time = time.time()

# upload PyRosetta Linux package into your google drive and put it path here
#pyrosetta_distr_path = google_drive.replace(' ', '\\ ') + '/PyRosetta/*'

# or alternatively you can download PyRosetta directly from GrayLab web site (but this might take some time!)
!mkdir /content/PyRosetta

# HERE, change user and password with the ones obtained through the PyRosetta license
# You might also need to change the release version of PyRosetta (currently 274 from Dec 13 2020)
!cd /content/PyRosetta && wget -O _PyRosetta_.tar.bz2 --user USERNAME --password PASSWORD http://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.MinSizeRel.python36.linux/PyRosetta4.MinSizeRel.python36.linux.release-274.tar.bz2   
pyrosetta_distr_path = '/content' + '/PyRosetta/_PyRosetta_.tar.bz2'

!mkdir _PyRosetta_
!tar xjf /content/PyRosetta/_PyRosetta_.tar.bz2 -C _PyRosetta_ --strip-components=1
!cd _PyRosetta_/setup && python setup.py install --install-lib=$pyrosetta_install_prefix_path
!rm -rf _PyRosetta_

import_end_time = time.time()

print(f'PyRosetta setup took: {import_end_time - import_start_time:.1f}s...')

In [ ]:
for e in os.listdir(pyrosetta_install_prefix_path):
    sys.path.append(pyrosetta_install_prefix_path + '/' + e)

Finally, we will perform some tests by importing PyRosetta modules to check that everything is working smoothly.

In [ ]:
from pyrosetta import *
pyrosetta.init()

In [ ]:
import pandas as pd
from pyrosetta import *
from rosetta.protocols.rosetta_scripts import *
from pyrosetta import (
    init, pose_from_sequence, pose_from_file, Pose, MoveMap, create_score_function, get_fa_scorefxn,
    MonteCarlo, TrialMover, SwitchResidueTypeSetMover, PyJobDistributor,
)

# Installation of GROMACS

## Downloading and installing GROMACS

We will first download the July 2020 version of GROMACS (ver. 2020.3) onto Google Colab and then install it. The installation takes **~40 min**.

**VERY IMPORTANT FIRST STEP:** Go To *Runtime* -> *Change Runtime Type* and select GPU!

In [ ]:
#Downloading GROMACS 2020.3
!wget http://ftp.gromacs.org/pub/gromacs/gromacs-2020.3.tar.gz

In [ ]:
#Extracting and Installing GROMACS 2020.3 onto a user-defined folder
%%bash
#First, we extract the downloaded folder containing GROMACS-2020.3
tar xfz gromacs-2020.3.tar.gz
echo "GROMACS extraction completed"
#Then, we create a "build" folder inside the extracted folder
cd gromacs-2020.3
mkdir build
#Next, we setup our GROMACS 2020.3 installation options, including GPU and a user-defined folder
cd build
cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=on -DCMAKE_INSTALL_PREFIX=/content/gromacs-2020
echo "GROMACS set up completed"
#Lastly, we compile GROMACS 2020.3 and install it inside the user-defined folder
make
echo "GROMACS building completed"
make check
echo "GROMACS testing completed"
make install
echo "GROMACS installation completed. Please check if any errors occurred during installation"

We now check that the installation was successful by loading the GROMACS PATH onto Google Colab.

In [ ]:
##Checking that GROMACS was successfully installed
%%bash
source /content/gromacs-2020/bin/GMXRC
gmx -h

##Backing up GROMACS on Google Drive

Once GROMACS is successfully compiled and installed on Google Colab, you can **save a back up** of GROMACS into your **Google Drive**. To do this, we first need to **Mount** your Google Drive into Google Colab to be able to transfer files, followed by backing up your GROMACS installation for future use, as follows:

In [ ]:
#First, we will mount your Google Drive to be able to transfer files
#You will be requested to authorize this Google Drive mounting
#By entering an authorization code into an input box and then pressing Enter
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Copying your compiled GROMACS to your Google Drive
#We will create and/or use the IBM3202 folder to create a folder for compiled programs
import os
import shutil
from pathlib import Path 
IBM3202 = Path("/content/gdrive/My Drive/IBM3202/")
if os.path.exists(IBM3202):
  print("IBM3202 already exists")
if not os.path.exists(IBM3202):
  os.mkdir(IBM3202)
  print("IBM3202 did not exists and was succesfully created")
#Then, we will copy the compiled GROMACS to this folder
shutil.copytree(str('/content/gromacs-2020'), str(IBM3202/'gromacs-2020'))
#!cp -d -r /content/gromacs-2020 "$IBM3202"/gromacs-2020
print("GROMACS successfully backed up!")

# Installation of SBM-enhanced GROMACS

## Downloading and installing SBM-enhanced GROMACS

We will first download the SBM-enhanced version of GROMACS (v5.0.4), available in the **Structure-based MOdels in Gromacs (SMOG)** [website](http://smog-server.org), onto Google Colab. Then we will compile and install it. The installation takes **~40 min**.

**NOTE:** In this case, you do not need GPU for compilation! Go To *Runtime* -> *Change Runtime Type* and make sure that _**None**_ is selected

In [ ]:
#Downloading SBM-enhanced GROMACS
!wget http://smog-server.org/extension/gmx-5.0.4_sbm8.1.tar.gz

In [ ]:
#Extracting and Installing SBM-enhanced GROMACS onto a user-defined folder
%%bash
#First, we extract the downloaded folder containing SBM-enhanced GROMACS
tar xfz gmx-5.0.4_sbm8.1.tar.gz
echo "SBM-enhanced GROMACS extraction completed"
#Then, we create a "build" folder inside the extracted folder
cd gmx5.0.4-sbm8.1
mkdir build
#Next, we setup our GROMACS installation options, including a user-defined folder
cd build
cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=off -DBUILD_SHARED_LIBS=off -DCMAKE_C_FLAGS=-DSBM -DCMAKE_CXX_FLAGS=-DSBM -DCMAKE_INSTALL_PREFIX=/content/gromacs-5.0.4-sbm
echo "SBM-enhanced GROMACS set up completed"
#Lastly, we compile GROMACS and install it inside the user-defined folder
make
echo "SBM-enhanced GROMACS building completed"
make install
echo "SBM-enhanced GROMACS installation completed. Please check if any errors occurred during installation"

We now check that the installation was successful by loading the GROMACS PATH onto Google Colab.

In [ ]:
##Checking that GROMACS was successfully installed
%%bash
source /content/gromacs-5.0.4-sbm/bin/GMXRC
gmx -h

##Backing up SBM-enhanced GROMACS on Google Drive

Once SBM-enhanced GROMACS is successfully compiled and installed on Google Colab, you can **save a back up** into your **Google Drive**. To do this, we first need to **Mount** your Google Drive into Google Colab to be able to transfer files, followed by backing up your SBM-enhanced GROMACS installation for future use, as follows:

In [ ]:
#First, we will mount your Google Drive to be able to transfer files
#You will be requested to authorize this Google Drive mounting
#By entering an authorization code into an input box and then pressing Enter
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Copying your compiled GROMACS to your Google Drive
#We will create and/or use the IBM3202 folder to create a folder for compiled programs
import os
import shutil
from pathlib import Path 
IBM3202 = Path("/content/gdrive/My Drive/IBM3202/")
if os.path.exists(IBM3202):
  print("IBM3202 already exists")
if not os.path.exists(IBM3202):
  os.mkdir(IBM3202)
  print("IBM3202 did not exists and was succesfully created")
#Then, we will copy the compiled GROMACS to this folder
shutil.copytree(str('/content/gromacs-5.0.4-sbm'), str(IBM3202/'gromacs-5.0.4-sbm'))
#!cp -d -r /content/gromacs-5.0.4-sbm "$IBM3202"/gromacs-5.0.4-sbm
print("GROMACS successfully backed up!")